## 010_an_merge
### Merge and clean jeopardy data sources 
### James Wilson

In [14]:
# -*- coding: utf-8 -*-
import pandas as pd
import re

In [15]:
# Load Data
archive_df = pd.read_csv("../Data/jeopardy_archive_data.csv")
twitter_df = pd.read_csv("../Data/jeopardy_twitter_data.csv")

In [16]:
# Reformat Data
# Melt twitter data 
twitter_df = pd.melt(twitter_df, id_vars =['id', 'created_at', 'favorite_count', 'retweet_count', 'text',
       'game_info_flg', 'Date'], var_name="answer_number")

In [17]:
# Add "Answer" variable to twitter 
    #IMPORTANT - reverse order of answers for archive data to match how twitter account was organized
archive_df["answer_number"] = ""
archive_df['dt_indx'] = archive_df.groupby(['Date']).cumcount()+1
archive_df.loc[archive_df["dt_indx"]==1,"answer_number"] = "Answer3"
archive_df.loc[archive_df["dt_indx"]==2,"answer_number"] = "Answer2"
archive_df.loc[archive_df["dt_indx"]==3,"answer_number"] = "Answer1"


In [18]:
# Merge Data Files
jeopardy = pd.merge(archive_df,twitter_df, on=['Date','answer_number'])
jeopardy.head(10)

,Nickname,Player Details,Final Score,Show Info,Full Name,Occupation,Hometown,Streak,Cash Winnings,Archive Info,Date,answer_number,dt_indx,id,created_at,favorite_count,retweet_count,text,game_info_flg,value
0,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$27,600","J! Archive - Show #8045, aired 2019-07-26",Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",NaN,NaN,J! Archive - Show #8045,2019-07-26,Answer3,1,1154934362737205253,2019-07-27 01:59:44,23,2,"7/26/19:\n""My medical school interview was wit...",True,"""A movie editor in France has a similar name t..."
1,Maggie,"Maggie Lehrman, an editor and writer from Broo...",$0,"J! Archive - Show #8045, aired 2019-07-26",Maggie Lehrman,an editor and writer,"Brooklyn, New York",NaN,NaN,J! Archive - Show #8045,2019-07-26,Answer2,2,1154934362737205253,2019-07-27 01:59:44,23,2,"7/26/19:\n""My medical school interview was wit...",True,"""The children's' book industry is phenomenal."""
2,Michael,"Michael Riggs, an educational therapist from T...",$2,"J! Archive - Show #8045, aired 2019-07-26",Michael Riggs,an educational therapist,"Tustin, California",NaN,NaN,J! Archive - Show #8045,2019-07-26,Answer1,3,1154934362737205253,2019-07-27 01:59:44,23,2,"7/26/19:\n""My medical school interview was wit...",True,"""My medical school interview was with a @Jeopa..."
3,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$4,400","J! Archive - Show #8044, aired 2019-07-25",Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",NaN,NaN,J! Archive - Show #8044,2019-07-25,Answer3,1,1154567652465106944,2019-07-26 01:42:33,6,1,"7/25/19:\n""I like cooking with my 2-year-old s...",True,"""It took me seconds to 'win' a game of anti-ch..."
4,Eric,"Eric Kaplan, a retired OB-GYN physician from L...",$0,"J! Archive - Show #8044, aired 2019-07-25",Eric Kaplan,a retired OB-GYN physician,"Long Beach, California",NaN,NaN,J! Archive - Show #8044,2019-07-25,Answer2,2,1154567652465106944,2019-07-26 01:42:33,6,1,"7/25/19:\n""I like cooking with my 2-year-old s...",True,"""Chuck Norris' aunt thought I looked justl ike..."
5,Shari,"Shari Meyer, a high school English teacher fro...",$0,"J! Archive - Show #8044, aired 2019-07-25",Shari Meyer,a high school English teacher,"Somerville, Massachusetts",NaN,NaN,J! Archive - Show #8044,2019-07-25,Answer1,3,1154567652465106944,2019-07-26 01:42:33,6,1,"7/25/19:\n""I like cooking with my 2-year-old s...",True,"""I like cooking with my 2-year-old son."""
6,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$30,000","J! Archive - Show #8043, aired 2019-07-24",Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",NaN,NaN,J! Archive - Show #8043,2019-07-24,Answer3,1,1154498659637051393,2019-07-25 21:08:24,8,3,7/24/19:\n“I played drums in a rhythmic troupe...,True,"“In Mexico, I was mistaken for Nicolas Cage.”"
7,Michelle,"Michelle Bruck, an attorney from Levittown, Pe...","$3,000","J! Archive - Show #8043, aired 2019-07-24",Michelle Bruck,an attorney,"Levittown, Pennsylvania",NaN,NaN,J! Archive - Show #8043,2019-07-24,Answer2,2,1154498659637051393,2019-07-25 21:08:24,8,3,7/24/19:\n“I played drums in a rhythmic troupe...,True,“I had to memorize the Gettysburg Address.”
8,Corin,"Corin Purifoy, a fiber artist from Milwaukee, ...","$6,100","J! Archive - Show #8043, aired 2019-07-24",Corin Purifoy,a fiber artist,"Milwaukee, Wisconsin",NaN,NaN,J! Archive - Show #8043,2019-07-24,Answer1,3,1154498659637051393,2019-07-25 21:08:24,8,3,7/24/19:\n“I played drums in a rhythmic troupe...,True,“I played drums in a rhythmic troupe.”
9,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$12,400","J! Archive - Show #8042, aired 2019-07-23",Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",NaN,NaN,J! Archive - Show #8042,2019-07-23,Answer3,1,1154153044772306945,2019-07-24 22:15:03,5,0,"7/23/19:\n""I like telling dad jokes.""\n""I run ...",True,"""My stuffed manatee is a comfort animal in my ..."


In [19]:
# Clean Data 
# clean money column
jeopardy['Final Score'] = jeopardy['Final Score'].str.replace(',', '')
jeopardy['Final Score'] = jeopardy['Final Score'].str.replace('$', '')
jeopardy['Final Score'] = jeopardy['Final Score'].astype(int)
        
# rename value column
jeopardy.rename(columns = {'value':'anecdote'}, inplace = True) 
jeopardy['anecdote'] = jeopardy['anecdote'].str.replace('"', '')

jeopardy_df = jeopardy.iloc[:,[4,2,5,6,10,11,12,15,16,19]]

In [20]:
# clean general clean_text 
# replace "&amp;" with "and" 
jeopardy_df['clean_text'] = jeopardy_df.loc[:,'anecdote'].str.replace('&amp;','and')


C:\Users\jwilson2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
# remove " " 
jeopardy_df.loc[:,'clean_text'] = jeopardy_df.loc[:,'clean_text'].str.replace('"','')
jeopardy_df.loc[:,'clean_text'] = jeopardy_df.loc[:,'clean_text'].str.replace("'",'')
jeopardy_df.loc[:,'clean_text'] = jeopardy_df.loc[:,'clean_text'].str.lstrip('\"')

# fix jeopardy exclamation mark
jeopardy_df.loc[:,'clean_text'] = jeopardy_df.loc[:,'clean_text'].str.replace('Jeopardy!','Jeopardy')
# general misspelling
jeopardy_df.loc[:,'clean_text'] = jeopardy_df.loc[:,'clean_text'].str.replace('justl ike','just like')


In [22]:
# rename individuals with @ signs 
# Function to clean the names 
def Clean_names(jprdy): 
    if re.search('@', jprdy): 
        sentence = re.sub( r"([A-Z])", r" \1", jprdy).split()
        jprdy = ' '.join(sentence)
        return jprdy
    else:
        return jprdy

# Updated the clean clean_text columns 
jeopardy_df.loc[:,'clean_text'] = jeopardy_df.loc[:,'clean_text'].apply(Clean_names) 

# remove '@'
jeopardy_df.loc[:,'clean_text'] = jeopardy_df.loc[:,'clean_text'].str.replace('@ ','')

In [23]:
# Clean hometown into city and state
jeopardy_df.loc[:,'city'] = ""
jeopardy_df.loc[:,'city'] = jeopardy_df.loc[:,'Hometown'].str.split(',',expand=True)[0]
jeopardy_df.loc[:,'city'] = jeopardy_df.loc[:,'city'].str.strip()

jeopardy_df.loc[:,'state'] = ""
jeopardy_df.loc[:,'state'] = jeopardy_df.loc[:,'Hometown'].str.split(',',expand=True)[1]
jeopardy_df.loc[:,'state'] = jeopardy_df.loc[:,'state'].str.strip()

In [24]:
# 


In [25]:
jeopardy_df.head(5)

,Full Name,Final Score,Occupation,Hometown,Date,answer_number,dt_indx,favorite_count,retweet_count,anecdote,clean_text,city,state
0,Jason Zuffranieri,27600,a math teacher,"Albuquerque, New Mexico",2019-07-26,Answer3,1,23,2,A movie editor in France has a similar name to...,A movie editor in France has a similar name to...,Albuquerque,New Mexico
1,Maggie Lehrman,0,an editor and writer,"Brooklyn, New York",2019-07-26,Answer2,2,23,2,The children's' book industry is phenomenal.,The childrens book industry is phenomenal.,Brooklyn,New York
2,Michael Riggs,2,an educational therapist,"Tustin, California",2019-07-26,Answer1,3,23,2,My medical school interview was with a @Jeopar...,My medical school interview was with a Jeopard...,Tustin,California
3,Jason Zuffranieri,4400,a math teacher,"Albuquerque, New Mexico",2019-07-25,Answer3,1,6,1,It took me seconds to 'win' a game of anti-chess.,It took me seconds to win a game of anti-chess.,Albuquerque,New Mexico
4,Eric Kaplan,0,a retired OB-GYN physician,"Long Beach, California",2019-07-25,Answer2,2,6,1,Chuck Norris' aunt thought I looked justl ike ...,Chuck Norris aunt thought I looked just like him.,Long Beach,California


In [26]:
jeopardy_df.to_csv('../Data/clean_jeopardy_data.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path